## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.


In [ ]:
%pip install okareo 

**Example 1: Create Scenario based on seed data passed as arguments**

In [1]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData, ScenarioType


OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Generate example scenario based on seed data and return results in one call
scenario_set_create = ScenarioSetCreate(name="My Test Scenario Set ",
                                        generation_type=ScenarioType.SEED,
                                        number_examples=1,
                                        seed_data=[SeedData(input_="example question or statement", result="example result"),
                                                   SeedData(input_="tell me about your capability", result="example result"),
                                                   SeedData(input_="what are your limitations", result="example result")])

source_scenario = okareo.create_scenario_set(scenario_set_create)

print(source_scenario.app_link)

https://app.okareo.com/project/f7bfcff0-2b8d-4fec-ab20-51669cd3e732/scenario/3335fbac-7f52-4fb5-935b-1e942fcce166


**Example 2: Create scenario set from a jsonl file upload with defined input and result fields**

In [9]:
import os
import tempfile
import json
from okareo import Okareo


# Step A - Create quick sample seed data file 
# Sample seed data jsonl file for classification model, need input and result fields at minimum
rows = [
    {
        "input": "I have some quality concerns with your product, who can I talk to?",
        "result": "complaints"
    },
    {
        "input": "The product is not working as expected, I'd like to return it?",
        "result": "returns",

    },
    {
        "input": "I'd like to purchase additional filters for my model, how much are they?",
        "result": "pricing",
    }
]

temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "seed_data_sample.jsonl")

# Write to a .jsonl file
with open(file_path, "w+") as file:
    for row in rows:
        file.write(json.dumps(row) + '\n')
    

# Step B - Create scenario set with seed data file
OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name="Sample Scenario Upload")
print(source_scenario) # use scenario set id in output for the next step

print(source_scenario.app_link)

# make sure to clean up tmp file
os.remove(file_path)

ScenarioSetResponse(scenario_id='12d9c263-508f-41c2-affd-5a7d273d063e', project_id='394c2c12-be7a-47a6-911b-d6c673bc543b', time_created=datetime.datetime(2024, 1, 18, 21, 7, 15, 661626), type='SEED', tags=None, name='Sample Scenario Upload', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})
https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/scenario/12d9c263-508f-41c2-affd-5a7d273d063e


**Example 3: Generate a new Scenario Set by referring to another scenario set as seed data**

In [10]:
import os
from okareo import Okareo


OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Use scenario set id from previous step as source for generation
generated_scenario = okareo.generate_scenarios(source_scenario=source_scenario,
                                     name="Generated Scenario Set",
                                     number_examples=2
                                     )

print(generated_scenario.app_link)

https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/scenario/b7f19b1f-3740-4d55-8dae-4541f060bd31


**Example 4: Generate a new Scenario Set using the Text Reverse Question generator. This will generate a new scenario with questions where the original input is the answer to the question generated.**

In [11]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

source_scenario = okareo.upload_scenario_set(file_path='./webbizz_10_articles.jsonl', scenario_name="Retrieval Articles Scenario")
generated_scenario_response = okareo.generate_scenarios(
    source_scenario=source_scenario,
    name="Questions from retrieval articles",
    number_examples=2,
    generation_type=ScenarioType.TEXT_REVERSE_QUESTION
)

print(generated_scenario_response)
print(generated_scenario_response.app_link)

ScenarioSetResponse(scenario_id='5ac4fa72-1fcf-4327-92cd-d5bea529ca83', project_id='394c2c12-be7a-47a6-911b-d6c673bc543b', time_created=datetime.datetime(2024, 1, 18, 21, 7, 47, 208386), type='TEXT_REVERSE_QUESTION', tags=['seed:dc7ebac1-1f19-45e1-ac3b-3dfb768e21d1'], name='Questions from retrieval articles', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})
https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/scenario/5ac4fa72-1fcf-4327-92cd-d5bea529ca83
